![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/2c54d005f76747fe83f77fbf8b3ec232).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Ejemplo predicción conjunto de test para envío a Kaggle
En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score, make_scorer, accuracy_score
from sklearn.model_selection import train_test_split

import re
import fasttext
import fasttext.util
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import nltk
from nltk.corpus import stopwords

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam, RMSprop
from keras import metrics
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.callbacks import EarlyStopping
from keras import backend as K
from livelossplot import PlotLossesKeras
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, ParameterGrid

import joblib

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [5]:
# Visualización datos de test
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [6]:
def obtener_embeddings(df, col_texto):

    vect = CountVectorizer(max_features=1000)
    X_dtm = vect.fit_transform(df[col_texto])
    
    return X_dtm
    

In [7]:
# Definición de variables predictoras (X)
X = obtener_embeddings(dataTraining, 'plot')

In [8]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [9]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X, y_genres, test_size=0.33, random_state=42)

In [10]:
# Definición y entrenamiento
clf = Sequential()
clf.add(Dense(350, input_dim=X.shape[1], activation='relu'))
clf.add(Dense(200, activation='relu'))
clf.add(Dense(y_genres.shape[1], activation='sigmoid'))  # Sigmoid para multietiqueta

clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
clf.fit(X_train, y_train_genres, epochs=40, batch_size=32, validation_split=0.2)

Epoch 1/40
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2082 - loss: 0.3748 - val_accuracy: 0.2382 - val_loss: 0.2645
Epoch 2/40
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3603 - loss: 0.2210 - val_accuracy: 0.3129 - val_loss: 0.2407
Epoch 3/40
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4230 - loss: 0.1683 - val_accuracy: 0.3327 - val_loss: 0.2391
Epoch 4/40
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4552 - loss: 0.1340 - val_accuracy: 0.3195 - val_loss: 0.2539
Epoch 5/40
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4664 - loss: 0.0998 - val_accuracy: 0.3015 - val_loss: 0.2866
Epoch 6/40
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4947 - loss: 0.0717 - val_accuracy: 0.3280 - val_loss: 0.3135
Epoch 7/40
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4892 - loss: 0.0479 - val_accuracy: 0.2845 - val_loss: 0.3530
Epoch 8/40
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5150 - loss: 0.0313 - val_accuracy: 0.

In [11]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


0.7889595481560642

In [12]:
# Lista de documentos
documents = ['Comedy movie']

# Inicializar CountVectorizer con el máximo de características deseado
vect = CountVectorizer(max_features=1000)

# Ajustar y transformar los documentos
X_dtm = vect.fit_transform(documents)

# predecir
clf.predict(X_dtm)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


array([[5.9417520e-02, 1.9667167e-04, 3.6141207e-03, 1.2557354e-02,
        6.2073863e-01, 8.4176309e-02, 1.0826041e-01, 5.1410813e-02,
        3.0985568e-03, 1.1664000e-02, 2.5773490e-05, 5.5544468e-04,
        2.4929070e-03, 2.7312136e-01, 3.3126995e-03, 1.2642150e-03,
        4.8276415e-04, 1.6993667e-03, 4.1617323e-02, 1.5361689e-03,
        1.8189577e-02, 1.1306871e-02, 1.2612280e-04, 2.2726268e-02]],
      dtype=float32)

In [17]:
joblib.dump(clf, './model_deployment/red_neuronal_peliculas_alterno.pkl', compress=3)

['./model_deployment/red_neuronal_peliculas_alterno.pkl']

In [14]:
def probabilidad_genero_pelicula(plot):

    #scaler = StandardScaler()

    dicc = {'plot': [plot]}
    
    base = pd.DataFrame(dicc)

    X_plot = obtener_embeddings(base, 'plot')
    
    cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']
    
    #clf = red_neuronal
    
    proba_genres = clf.predict(X_plot)
    df = pd.DataFrame(proba_genres, columns=cols)
    diccionario = df.to_dict(orient='list')

    return diccionario

In [15]:
clf

<Sequential name=sequential, built=True>

In [19]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vect.fit_transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf.predict(X_test_dtm)

106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step


In [22]:
    # Lista de documentos
documents = ['comedy movie']

    # Inicializar CountVectorizer con el máximo de características deseado
vect2 = CountVectorizer(max_features=1000)

    # Ajustar y transformar los documentos
plotFit = vect2.fit_transform(documents)

clf.predict(plotFit)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([[5.9417520e-02, 1.9667167e-04, 3.6141207e-03, 1.2557354e-02,
        6.2073863e-01, 8.4176309e-02, 1.0826041e-01, 5.1410813e-02,
        3.0985568e-03, 1.1664000e-02, 2.5773490e-05, 5.5544468e-04,
        2.4929070e-03, 2.7312136e-01, 3.3126995e-03, 1.2642150e-03,
        4.8276415e-04, 1.6993667e-03, 4.1617323e-02, 1.5361689e-03,
        1.8189577e-02, 1.1306871e-02, 1.2612280e-04, 2.2726268e-02]],
      dtype=float32)

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_RF.csv', index_label='ID')
res.head()